# Download Data

In [1]:
# The code was removed by Watson Studio for sharing.

In [4]:
!rm data_training.csv
!ls

11269.png      13568.png   243.png     6324.png  6569.png   8058.png
1146.png       137201.png  24529.png   6325.png  67180.png  887.png
11473.png      140746.png  24547.png   6329.png  674.png    931.png
11638.png      14917.png   24823.png   6335.png  69319.png  996.png
11646.png      176.png	   3283.png    6360.png  7845.png   data.csv
12228.png      177.png	   33558.png   6375.png  784.png    data_processed.csv
123257.png     222.png	   5206.png    6386.png  7929.png   label_encoder.pkl
127602.png     240.png	   552549.png  6393.png  8033.png
135504501.png  241.png	   6115.png    6410.png  8053.png


In [6]:
download_file_cos(credentials_1, 'data_training.csv', 'data_training.csv')
!ls

File Downloaded
11269.png      13568.png   243.png     6324.png  6569.png   8058.png
1146.png       137201.png  24529.png   6325.png  67180.png  887.png
11473.png      140746.png  24547.png   6329.png  674.png    931.png
11638.png      14917.png   24823.png   6335.png  69319.png  996.png
11646.png      176.png	   3283.png    6360.png  7845.png   data.csv
12228.png      177.png	   33558.png   6375.png  784.png    data_processed.csv
123257.png     222.png	   5206.png    6386.png  7929.png   data_training.csv
127602.png     240.png	   552549.png  6393.png  8033.png   label_encoder.pkl
135504501.png  241.png	   6115.png    6410.png  8053.png


In [7]:
!head -10 data_training.csv

pairs_enc,distance,bond
0,1.5119999999999998,1
2,1.0940555607463451,1
2,1.0940439342183659,1
2,1.0940911982097288,1
0,1.5119999999999998,0
2,1.0940439342183659,1
2,1.0940911982097288,1
2,1.0940555607463451,1
2,1.0940911982097288,0


# Train,Val,Test Split

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

In [15]:
x=pd.read_csv('data_training.csv')

In [18]:
x.head()

,pairs_enc,distance,bond
0,0,1.512000,1
1,2,1.094056,1
2,2,1.094044,1
3,2,1.094091,1
4,0,1.512000,0


In [19]:
x.shape

(1062, 3)

In [16]:
x.columns

Index(['pairs_enc', 'distance', 'bond'], dtype='object')

In [17]:
y=x['bond']

In [21]:
x=x.drop('bond',axis=1)

In [22]:
x.head()

,pairs_enc,distance
0,0,1.512000
1,2,1.094056
2,2,1.094044
3,2,1.094091
4,0,1.512000


In [23]:
y.head()

0    1
1    1
2    1
3    1
4    0
Name: bond, dtype: int64

In [24]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3)

In [26]:
print('x_train: {}  y_train: {}\nx_val: {}  y_val: {}'.format(x_train.shape,y_train.shape,x_val.shape,y_val.shape))

x_train: (743, 2)  y_train: (743,)
x_val: (159, 2)  y_val: (159,)


In [25]:
x_val,x_test,y_val,y_test=train_test_split(x_val,y_val,test_size=0.5)

In [27]:
print('x_val: {}  y_val: {}\nx_test: {}  y_test: {}'.format(x_val.shape,y_val.shape,x_test.shape,y_test.shape))

x_val: (159, 2)  y_val: (159,)
x_test: (160, 2)  y_test: (160,)


In [28]:
y_val.value_counts()

0    87
1    72
Name: bond, dtype: int64

In [29]:
y_test.value_counts()

1    89
0    71
Name: bond, dtype: int64

In [30]:
y_train.value_counts()

0    373
1    370
Name: bond, dtype: int64

# Model

In [12]:
tf.__version__

'1.13.1'

## 1. NN

In [79]:
input_shape=(2,)
model_nn=keras.Sequential()
model_nn.add(keras.layers.Dense(512,activation='relu',input_shape=input_shape))
model_nn.add(keras.layers.Dense(512,activation='relu'))
model_nn.add(keras.layers.Dense(256,activation='relu'))
model_nn.add(keras.layers.Dense(256,activation='relu'))
model_nn.add(keras.layers.Dense(128,activation='relu'))
model_nn.add(keras.layers.Dense(100,activation='relu'))
model_nn.add(keras.layers.Dense(64,activation='relu'))
model_nn.add(keras.layers.Dense(1,activation='sigmoid'))

In [80]:
model_nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 512)               1536      
_________________________________________________________________
dense_53 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_54 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_55 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_56 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_57 (Dense)             (None, 100)               12900     
_________________________________________________________________
dense_58 (Dense)             (None, 64)                6464      
__________

In [83]:
#help(model_nn.compile)

In [84]:
model_nn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [85]:
model_nn.fit(x_train,y_train,batch_size=256,epochs=200,validation_data=(x_val,y_val))

Train on 743 samples, validate on 159 samples
Epoch 1/200
743/743 [==============================] - 3s 3ms/sample - loss: 0.6573 - acc: 0.5209 - val_loss: 0.5762 - val_acc: 0.5723
Epoch 2/200
743/743 [==============================] - 1s 858us/sample - loss: 0.5743 - acc: 0.5935 - val_loss: 0.5367 - val_acc: 0.7296
Epoch 3/200
743/743 [==============================] - 1s 831us/sample - loss: 0.5564 - acc: 0.7174 - val_loss: 0.5234 - val_acc: 0.7358
Epoch 4/200
743/743 [==============================] - 1s 861us/sample - loss: 0.5312 - acc: 0.7308 - val_loss: 0.5107 - val_acc: 0.7233
Epoch 5/200
743/743 [==============================] - 1s 944us/sample - loss: 0.5217 - acc: 0.7308 - val_loss: 0.4888 - val_acc: 0.7296
Epoch 6/200
743/743 [==============================] - 1s 1ms/sample - loss: 0.4942 - acc: 0.7308 - val_loss: 0.4724 - val_acc: 0.8050
Epoch 7/200
743/743 [==============================] - 1s 994us/sample - loss: 0.4711 - acc: 0.8237 - val_loss: 0.4480 - val_acc: 0.8239

In [86]:
model_nn.evaluate(x_test,y_test)

160/160 [==============================] - 0s 1ms/sample - loss: 0.2664 - acc: 0.8938


[0.2664415568113327, 0.89375]

## Random Forest

In [87]:
from sklearn.ensemble import RandomForestClassifier

In [88]:
model_rf=RandomForestClassifier()
model_rf.fit(x_train,y_train)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [89]:
from sklearn.metrics import accuracy_score

In [94]:
y_pred=model_rf.predict(x_test)

In [95]:
accuracy_score(y_test,y_pred)

0.83125

## GBM, SVM, Logistic Regression

### Hyperparameter tuning

# Serialize Models

In [96]:
#model_nn.save('connectivity_model_nn_02.h5')
"""
Did not work after importing in another notebook
"""

In [103]:
#This method worked
tf.keras.models.save_model(model_nn,'connectivity_model_nn_02.h5',overwrite=True,include_optimizer=True)

In [104]:
!ls

11269.png      140746.png  3283.png    6386.png   8053.png
1146.png       14917.png   33558.png   6393.png   8058.png
11473.png      176.png	   5206.png    6410.png   887.png
11638.png      177.png	   552549.png  6569.png   931.png
11646.png      222.png	   6115.png    67180.png  996.png
12228.png      240.png	   6324.png    674.png	  connectivity_model_nn_02.h5
123257.png     241.png	   6325.png    69319.png  data.csv
127602.png     243.png	   6329.png    7845.png   data_processed.csv
135504501.png  24529.png   6335.png    784.png	  data_training.csv
13568.png      24547.png   6360.png    7929.png   label_encoder.pkl
137201.png     24823.png   6375.png    8033.png


# Upload to IBM Object Store

## 1. Current Project bucket

In [105]:
# The code was removed by Watson Studio for sharing.

In [106]:
upload_file_cos(credentials_1,'connectivity_model_nn_02.h5','connectivity_model_nn_02.h5')

File Uploaded


## 2. Coursera/Kaggle project bucket

In [107]:
# The code was removed by Watson Studio for sharing.

In [108]:
upload_file_cos(credentials_2,'connectivity_model_nn_02.h5','connectivity_model_nn_02.h5')

File Uploaded
